## Generative Models/Mixture Models - Expectation Maximization

In [ ]:
import math
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# calculate the likelihood of a sample given the parameters of the gaussian
def gaussian(mean: float, std: float, x: np.array):
    if abs(std) < 1e-12: std = 1e-12
    return 1/(std*np.sqrt(2*math.pi))*np.exp(-0.5*((x-mean)/std)**2)

In [ ]:
# calculate the posterior of sample belonging to a distribution
def marginal_likelihood(likelihoods: np.array, priors: np.array):
        return np.sum(np.multiply(likelihoods, priors))

In [ ]:
# calculate the posterior of sample belonging to a distribution
def posterior(likelihood: float, prior: float, marginal_likelihood: float):
        return likelihood*prior/marginal_likelihood

In [ ]:
class EM:
    def __init__(self, n_components: int, samples: np.array, labeled_samples: np.array, labeled = False, n_iterations: int = 10):
        self.n_components = n_components
        self.n_iterations = n_iterations
        self.samples = samples
        self.labeled_samples = labeled_samples         
        
        # stds for each component
        self.stds = np.ones(n_components)
        # means for each component - sampling uniform random
        self.means = np.random.uniform(low=np.min(samples), high=np.max(samples), size=n_components)
        if labeled:
            self.means = np.mean(labeled_samples, axis=1)
        
        # priors of the sample being from the respective component
        self.priors = np.ones(n_components)/n_components
        # likelihoods of the sample belonging to the respective component
        self.lhoods = np.zeros([n_components,len(samples)])
        # posteriors of the sample being from the respective component
        self.posts = np.zeros([n_components,len(samples)])
        
        self.fig, self.axs = plt.subplots(n_iterations, figsize=[15,n_iterations*3])
               
    def fit(self):
        for i in range(self.n_iterations):
            self.plot(i)        
            # caculate the likelihood of the sample belonging to the respective component
            for c in range(self.n_components):
                self.lhoods[c,:] = gaussian(self.means[c], self.stds[c], self.samples)
            
            # caculate the posterior of the sample belonging to the respective component
            for c in range(self.n_components):
                for s in range(len(self.samples)):
                    marginal = marginal_likelihood(self.lhoods[:,s], self.priors)
                    self.posts[c,s] = posterior(self.lhoods[c,s],self.priors[c], marginal)
            self.priors = np.sum(self.lhoods, axis=1)

            # caculate the new mean and variance
            for c in range(self.n_components):
                self.means[c] = np.sum(np.multiply(self.posts[c,:],self.samples))/np.sum(self.posts[c,:])
                self.stds[c] = np.sum(np.multiply(self.posts[c,:],np.square(self.samples-self.means[c])))/np.sum(self.posts[c,:])
                
    def plot(self, i):
        clrs = np.array(['r', 'b', 'g'])
        min_val = np.min(self.samples)
        max_val = np.max(self.samples)
        x = np.linspace(min_val, max_val, 1000)
        
        labels = np.argmax(self.posts,axis=0)
        col = np.where(labels==0,clrs[0],np.where(labels==1,clrs[1],clrs[2]))
        
        self.axs[i].scatter(self.samples, np.zeros(len(self.samples)), c=col)

                
        for c in range(self.n_components):
            self.axs[i].plot(x, gaussian(self.means[c], self.stds[c], x), c=clrs[c])
            #print(f"mean of component {c} is {self.means[c]}")
            #print(f"std  of component {c} is {self.stds[c]}")

In [ ]:
n_samples = 10

n_labled = 2
# create three gaussian distributions
class_a = np.random.normal(-5, 1, n_samples)
class_b = np.random.normal(0, 1, n_samples)
class_c = np.random.normal(5, 1, n_samples)
# random number generator
rng = np.random.default_rng()
# sample labeled data
labeled_a = rng.choice(class_a,size=n_labled)
labeled_b = rng.choice(class_b,size=n_labled)
labeled_c = rng.choice(class_c,size=n_labled)

labeled_samples = np.array([labeled_a, labeled_b, labeled_c])

samples = np.concatenate((class_a, class_b, class_c))

### Data generated from Three Gaussians

Gaussians consist of:
* mean
* variance

In [ ]:
# data distribution of the samples
plt.scatter(class_a, np.zeros(n_samples), facecolors='none', edgecolors='black')
plt.scatter(class_b, np.zeros(n_samples), facecolors='none', edgecolors='black')
plt.scatter(class_c, np.zeros(n_samples), facecolors='none', edgecolors='black')

plt.scatter(labeled_a, np.zeros(n_labled), c="red")
plt.scatter(labeled_b, np.zeros(n_labled), c="blue")
plt.scatter(labeled_c, np.zeros(n_labled), c="green");

### Expectation Maximization
* place n Gaussians
* caculate the likelihood of the sample belonging to the respective component
* caculate the posterior of the sample belonging to the respective component
* $Posterior \propto Likelihood \times Prior$


In [ ]:
model = EM(3, samples, labeled_samples, False, 4)

model.fit()